# Adding the weather to the dataset 
For each city in the venues dataset, we would need to get the corresponding daily weather data. 
## Imports

In [3]:
# imports
import pandas as pd 
import numpy as np

## Downloaded packages
If you don't have the following libraries installed, uncomment the pip call run the following cells. It will later be used to get the coordinates of cities needed to find weather data. 

In [ ]:
#!pip install geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

In [ ]:
#!pip install openmeteo-requests requests-cache retry-requests
import openmeteo_requests
import requests_cache
from retry_requests import retry

In [6]:
# load the venues dataset
venues = pd.read_csv("./datasets/venues.csv") # path to venues data

In [25]:
# how many unique cities are there? 
venues['city'] = venues['city'].str.lower()
print(venues[['city', 'country']].drop_duplicates())

             city country
0     albuquerque      US
1        brooklyn      US
2         windsor      CA
3         burnaby      CA
4         tisdale      CA
..            ...     ...
591         unity      CA
592    white rock      CA
593      coboconk      CA
597    richardson      US
599   mississauga      CA

[387 rows x 2 columns]


In [8]:
# validating that the countries are just CA and US
venues[['country']].drop_duplicates()

country
0      US
2      CA

In [9]:
# load the daily transactions data set 
dailyTransactions = pd.read_csv("./datasets/dailyTransactions.csv")

In [10]:
# check for null venue ids 
dailyTransactions['venue_xref_id'].isnull().sum()
dailyTransactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92365 entries, 0 to 92364
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   venue_xref_id                   92365 non-null  object 
 1   business_date                   92365 non-null  object 
 2   bill_total_billed               92365 non-null  float64
 3   bill_total_discount_item_level  92365 non-null  float64
 4   bill_total_gratuity             92365 non-null  float64
 5   bill_total_net                  92365 non-null  float64
 6   bill_total_tax                  92365 non-null  float64
 7   bill_total_voided               92365 non-null  float64
 8   payment_amount                  92365 non-null  float64
 9   payment_count                   92365 non-null  int64  
 10  payment_total_tip               92365 non-null  float64
 11  sales_revenue_with_tax          92365 non-null  float64
 12  order_duration_seconds          

In [11]:
# check for venue id duplicates before merging with daily transaction data
venues['venue_xref_id'].duplicated().sum()

0

In [12]:
# merge the two datasets
venue_transaction = pd.merge(dailyTransactions, venues, on='venue_xref_id', how='inner') 

In [13]:
# check the length of the table vs the original daily transaction summary data
len(dailyTransactions)

92365

## Inserting the weather data 
We will be using an open weather api to get the weather data. https://open-meteo.com/
This api only works in longitudes and latitudes for the requests, so we will get those values for the cities in the venue dataset. 
### Inserting latitudes and longitudes 

In [ ]:
# # get list of cities to search for
# city_df = venues[['city', 'country']].drop_duplicates().reset_index()
# # initiate cols 
# city_df['latitude'] = np.nan
# city_df['longitude'] = np.nan
# city_df.to_csv("./datasets/cities.csv", index=True)

In [15]:
geolocator = Nominatim(user_agent="GetCoordsApp")

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)

# functino that gets the latitude and longitude based on city, country 
def get_coords(city, country):
    country_full = "Canada"
    if (country == "US"): country_full = "United States of America"
    try:
        location = geolocator.geocode(f"{city}, {country_full}")
        if location: 
            print([location.latitude, location.longitude])
            return [location.latitude, location.longitude]
        else: 
            print("No location found")
            return None 
    except Exception as e: 
        print(f"Error: {e}")
        raise e

In [ ]:
city_df = pd.read_csv('./datasets/cities.csv')
df_length = len(city_df)
for i in range(0, df_length): 
    row = city_df.iloc[i]
    if (not pd.isna(row['latitude'])): continue # already filled in 
    try: 
        location = get_coords(row['city'], row['country'])
        if (location is None): continue 
        city_df.loc[i, 'latitude'] = location[0]
        city_df.loc[i, 'longitude'] = location[1]
    except Exception as e: 
        print(f"Error: {e}")
        # save a backup of the current state 
        city_df.to_csv('./datasets/cities.csv')
        break
city_df.to_csv('./datasets/cities.csv', index=False)
city_df = pd.read_csv('./datasets/cities.csv')


[32.7762719, -96.7968559]
[32.7174202, -117.162772]
[45.1404664, -89.1524959]
[50.04303, -110.679016]
[39.1663259, -119.7670374]
[41.779588, -71.4366813]
[40.6867667, -73.3734547]
[21.418555, -157.804184]
[47.2455013, -122.438329]
[49.0846258, -123.057938]
[48.7544012, -122.478836]
[34.0133561, -117.690075]
[47.4892246, -117.579304]
[49.163168, -123.137414]
[34.5960774, -120.1381144]
No location found
[41.9426657, -70.7618592]
[41.616016, -87.5295837]
[41.5156358, -88.1452045]
[43.6660467, -98.1383168]
[53.8242258, -113.3282658]
[32.4490141, -81.7832912]
[47.7815088, -65.7175991]
[47.6208671, -65.6537546]
[52.4428508, -109.1548985]
[49.0235357, -122.797924]
[44.658576, -78.798191]
[40.1194651, -95.7607517]
[43.5896231, -79.6443879]


### Checking the dataset 

In [40]:
# check for null coordinates 
city_df = pd.read_csv('./datasets/cities.csv')
city_df.loc[city_df['latitude'].isna()]

city country  latitude  longitude
137   whitchurch-stoufville      CA       NaN        NaN
291  st. andrews by the sea      CA       NaN        NaN
373             scaroborogh      CA       NaN        NaN

Since these cities can be located, we will manually add in the coordinates

### Using Weather API to Obtain Weather Data
As mentionned, we will be using https://open-meteo.com/ for the weather data. To minimize the number of API calls, let's find out what date range is needed for each city/location. 

In [ ]:
# what are the earliest and latest dates for the entire dataset? 
print(venue_transaction['business_date'].min())
print(venue_transaction['business_date'].max())

2024-07-01
2024-12-31


In [52]:
# using the provided code from open-meteo

def getWeatherData(row): 
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)
    url = "https://archive-api.open-meteo.com/v1/archive"
    
    params = {
	"latitude": row['latitude'],
	"longitude": row['longitude'],
	"start_date": "2024-07-01",
	"end_date": "2024-12-31",
	"daily": ["apparent_temperature_max", "apparent_temperature_min", "daylight_duration", "precipitation_sum", "rain_sum", "snowfall_sum"]
    }
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    print(response)
    daily = response.Daily()
    daily_apparent_temperature_max = daily.Variables(0).ValuesAsNumpy()
    daily_apparent_temperature_min = daily.Variables(1).ValuesAsNumpy()
    daily_daylight_duration = daily.Variables(2).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(3).ValuesAsNumpy()
    daily_rain_sum = daily.Variables(4).ValuesAsNumpy()
    daily_snowfall_sum = daily.Variables(5).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}

    daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
    daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
    daily_data["daylight_duration"] = daily_daylight_duration
    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["rain_sum"] = daily_rain_sum
    daily_data["snowfall_sum"] = daily_snowfall_sum

    daily_dataframe = pd.DataFrame(data = daily_data)
    daily_dataframe['city'] = row['city']
    daily_dataframe['country'] = row['country'] 
    daily_dataframe['latitude'] = row['latitude']
    daily_dataframe['longitude'] = row['longitude'] 

    cols_to_move = ['city', 'country', 'latitude', 'longitude']
    remaining_cols = [col for col in daily_dataframe.columns if col not in cols_to_move]

    new_order = cols_to_move + remaining_cols
    daily_dataframe = daily_dataframe[new_order]

    return daily_dataframe

# getWeatherData({'city': 'Markham', 'country': 'CA', 'latitude': 43.8563707, 'longitude': -79.3376825}) # test 

In [ ]:
city_df = pd.read_csv('./datasets/cities.csv')
city_daily_weather = pd.DataFrame()

for i in range(0, len(city_df)): 
    print(i)
    df = getWeatherData(city_df.iloc[i])
    city_daily_weather = pd.concat([city_daily_weather, df], ignore_index=True)
    time.sleep(1)
    city_daily_weather.to_csv('./datasets/cityWeather.csv', index=False)
city_daily_weather

In [62]:
# # export backup 
# city_daily_weather = pd.read_csv('./datasets/cityWeather.csv')
# city_daily_weather = city_daily_weather[['city', 'country','latitude','longitude','date','apparent_temperature_max','apparent_temperature_min','daylight_duration','precipitation_sum','rain_sum','snowfall_sum']]
# city_daily_weather.to_csv('./datasets/cityWeather.csv', index=False)


In [ ]:
# link the weather to the daily summary and venues dataframe
def getDate(datetime):
    return datetime.split(' ')[0] # get just the date, no time
city_daily_weather['business_date'] = city_daily_weather['date'].apply(getDate)

In [71]:
venue_transaction_weather = pd.merge(venue_transaction, city_daily_weather, on=['city', 'country', 'business_date'], how="inner")

In [73]:
# Export the data set 
venue_transaction_weather.to_csv('./datasets/venue_daily_weather.csv', index=False)